# DuckDB benchmark - Azure vs AWS 

Benchmarking the wall-clock time to find the nearest point in a 12 mln row GeoParquet dataset that consists of several partitions

## Azure

In [29]:
%%time
import os
import pathlib

import dotenv
import duckdb
import geopandas as gpd

dotenv.load_dotenv(override=True)
azure_connection_string = os.getenv("AZURE_CONNECTION_STRING")

con = duckdb.connect(database=":memory:", read_only=False)

# Install and load the spatial extension
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")
con.execute("LOAD azure;")
con.execute(f"SET azure_storage_connection_string = '{azure_connection_string}';")

# # Path to your Parquet file(s)
transects_href = "azure://transects/gcts-2000m.parquet/**/*.parquet"

# Specify your point of interest (longitude, latitude)
point_x, point_y = -4.29, 52.19  # Example coordinates
point_wkt = (
    gpd.GeoSeries.from_xy([point_x], [point_y], crs=4326).to_crs(3857).to_wkt().item()
)
your_radius = 10000.0  # Ensure radius is treated as DOUBLE

# Adjust this query with your actual geometry column name and Parquet file path
query = f"""
SELECT *, ST_Distance(ST_GeomFromWKB(geometry), ST_GeomFromText('{point_wkt}')) AS distance
FROM '{transects_href}'
WHERE ST_DWithin(ST_GeomFromWKB(geometry), ST_GeomFromText('{point_wkt}'), {your_radius})
ORDER BY distance
LIMIT 1;
"""

# Execute the query and fetch the result as a DataFrame
nearest_result = con.execute(query).fetchdf()

# Display the nearest geometry and its distance from the input point
print(nearest_result)

# Close the DuckDB connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

            tr_name       lon        lat    bearing  utm_crs  coastline_name  \
0  cl31596tr1519446 -4.317812  52.216587  326.57959    32630           31596   

                                            geometry  __null_dask_index__  \
0  [1, 2, 0, 0, 0, 2, 0, 0, 0, 85, 239, 221, 110,...              3726912   

  quadkey     distance  
0   qk031  4105.767123  
CPU times: user 37.9 s, sys: 2.93 s, total: 40.8 s
Wall time: 1min 39s


## AWS S3

In [36]:
%%time
import os
import pathlib

import dotenv
import duckdb
import geopandas as gpd

dotenv.load_dotenv(override=True)
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

con = duckdb.connect(database=":memory:", read_only=False)


# Install and load the spatial extension
con.execute("LOAD spatial;")
con.execute("LOAD httpfs;")
con.execute(f"SET s3_region = 'eu-west-2';")
con.execute(f"SET s3_access_key_id = '{aws_access_key_id}';")
con.execute(f"SET s3_secret_access_key = '{aws_secret_access_key}';")

# # Path to your Parquet file(s)
transects_href = "s3://coastmonitor/gcts-2000m.parquet/**/*.parquet"

# Specify your point of interest (longitude, latitude)
point_x, point_y = -4.29, 52.19  # Example coordinates
point_wkt = (
    gpd.GeoSeries.from_xy([point_x], [point_y], crs=4326).to_crs(3857).to_wkt().item()
)
your_radius = 10000.0  # Ensure radius is treated as DOUBLE

# Adjust this query with your actual geometry column name and Parquet file path
query = f"""
SELECT *, ST_Distance(ST_GeomFromWKB(geometry), ST_GeomFromText('{point_wkt}')) AS distance
FROM '{transects_href}'
WHERE ST_DWithin(ST_GeomFromWKB(geometry), ST_GeomFromText('{point_wkt}'), {your_radius})
ORDER BY distance
LIMIT 1;
"""

# Execute the query and fetch the result as a DataFrame
nearest_result = con.execute(query).fetchdf()

# Display the nearest geometry and its distance from the input point
print(nearest_result)

# Close the DuckDB connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

            tr_name       lon        lat    bearing  utm_crs  coastline_name  \
0  cl31596tr1519446 -4.317812  52.216587  326.57959    32630           31596   

                                            geometry  __null_dask_index__  \
0  [1, 2, 0, 0, 0, 2, 0, 0, 0, 85, 239, 221, 110,...              3726912   

  quadkey     distance  
0   qk031  4105.767123  
CPU times: user 35.2 s, sys: 3.53 s, total: 38.7 s
Wall time: 1min 41s
